<a href="https://colab.research.google.com/github/GittleBittle/bread-and-butter/blob/main/CifarGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1) Importing Python Packages for GAN**


In [ ]:
from keras.datasets import cifar10, mnist
from keras.models import Sequential
from keras.layers import Reshape
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import Dropout
from keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.optimizers import Adam
import numpy as np
!mkdir generated_images

mkdir: cannot create directory ‘generated_images’: File exists


## **2) Parameters for Neural Networks & Data**

In [ ]:
img_width = 32
img_height = 32
channels = 3
img_shape = (img_width, img_height, channels)
latent_dim = 300
adam = Adam(lr=0.0002)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


## **3) Building Generator**





In [ ]:
from keras.backend import conv2d
from numpy.core.fromnumeric import reshape
def build_generator():
    model = Sequential()
    model.add(Dense(256 * 4 * 4, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((4,4,256)))

    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add((Conv2D(3, (3,3), activation='tanh', padding='same')))

    model.summary()
    return model

generator = build_generator()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 4096)              1232896   
                                                                 
 leaky_re_lu_51 (LeakyReLU)  (None, 4096)              0         
                                                                 
 reshape_9 (Reshape)         (None, 4, 4, 256)         0         
                                                                 
 conv2d_transpose_27 (Conv2D  (None, 8, 8, 128)        524416    
 Transpose)                                                      
                                                                 
 leaky_re_lu_52 (LeakyReLU)  (None, 8, 8, 128)         0         
                                                                 
 conv2d_transpose_28 (Conv2D  (None, 16, 16, 128)      262272    
 Transpose)                                          

## **4) Building Discriminator**

In [ ]:
def build_discriminator():
  model = Sequential()
  model.add(Conv2D(64, (3,3), padding='same', input_shape=img_shape))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2D(64, (3,3), padding='same', input_shape=img_shape))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2D(128, (3,3), padding='same', input_shape=img_shape))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2D(128, (3,3), padding='same', input_shape=img_shape))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2D(256, (3,3), padding='same'))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2D(256, (3,3), padding='same'))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2D(256, (3,3), padding='same'))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Flatten())
  model.add(Dropout(0.4))
  model.add(Dense(1, activation='sigmoid'))

  model.summary()
  return model

discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_19 (Conv2D)          (None, 32, 32, 64)        1792      
                                                                 
 leaky_re_lu_55 (LeakyReLU)  (None, 32, 32, 64)        0         
                                                                 
 conv2d_20 (Conv2D)          (None, 32, 32, 64)        36928     
                                                                 
 leaky_re_lu_56 (LeakyReLU)  (None, 32, 32, 64)        0         
                                                                 
 conv2d_21 (Conv2D)          (None, 32, 32, 128)       73856     
                                                                 
 leaky_re_lu_57 (LeakyReLU)  (None, 32, 32, 128)       0         
                                                                 
 conv2d_22 (Conv2D)          (None, 32, 32, 128)     

## **5) Connecting Neural Networks to build GAN**

In [ ]:
GAN = Sequential()
discriminator.trainable = False
GAN.add(generator)
GAN.add(discriminator)

GAN.compile(loss='binary_crossentropy', optimizer=adam)

In [ ]:
#GAN.summary()

## **6) Outputting Images**


In [ ]:
#@title
## **7) Outputting Images**
import matplotlib.pyplot as plt
import glob
import imageio
import PIL

save_name = 0.00000000

def save_imgs(epoch):
    r, c = 2, 2
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)
    global save_name
    save_name += 0.00000001
    # print("%.8f" % save_name)

    # Rescale images 0 - 1
    # gen_imgs = 0.5 * gen_imgs + 0.5
    gen_imgs = (gen_imgs + 1) / 2.0
    # gen_imgs = gen_imgs * 255

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("generated_images/%.8f.png" % save_name)
    plt.close()

## **7) Training GAN**

In [ ]:
def train(epochs, batch_size=64, save_interval=200):
  (X_train, _), (_, _) = cifar10.load_data()

  # print(X_train.shape)
  #Rescale data between -1 and 1
  X_train = X_train / 127.5 -1.
  bat_per_epo = int(X_train.shape[0] / batch_size)
  # X_train = np.expand_dims(X_train, axis=3)
  # print(X_train.shape)

  #Create our Y for our Neural Networks
  valid = np.ones((batch_size, 1))
  fakes = np.zeros((batch_size, 1))

  for epoch in range(epochs):
    for j in range(bat_per_epo):
    #Get Random Batch
      idx = np.random.randint(0, X_train.shape[0], batch_size)
    imgs = X_train[idx]

    #Generate Fake Images
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    gen_imgs = generator.predict(noise)

    #Train discriminator
    d_loss_real = discriminator.train_on_batch(imgs, valid)
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fakes)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    
    #inverse y label
    g_loss = GAN.train_on_batch(noise, valid)

    print("******* %d %d [D loss: %f, acc: %.2f%%] [G loss: %f]" % (epoch, j, d_loss[0], 100* d_loss[1], g_loss))

    if(epoch % save_interval) == 0:
      save_imgs(epoch)


train(30000, batch_size=64, save_interval=200)

******* 0 780 [D loss: 0.164041, acc: 92.97%] [G loss: 6.614654]
******* 1 780 [D loss: 0.114996, acc: 94.53%] [G loss: 7.548945]
******* 2 780 [D loss: 0.115054, acc: 96.09%] [G loss: 6.486835]
******* 3 780 [D loss: 0.216823, acc: 89.06%] [G loss: 6.403454]
******* 4 780 [D loss: 0.168672, acc: 92.97%] [G loss: 6.212219]
******* 5 780 [D loss: 0.134640, acc: 95.31%] [G loss: 7.452126]
******* 6 780 [D loss: 0.162966, acc: 93.75%] [G loss: 7.844760]
******* 7 780 [D loss: 0.137042, acc: 94.53%] [G loss: 7.259841]
******* 8 780 [D loss: 0.158641, acc: 95.31%] [G loss: 7.466525]
******* 9 780 [D loss: 0.160528, acc: 96.09%] [G loss: 6.846779]
******* 10 780 [D loss: 0.148726, acc: 92.19%] [G loss: 6.480263]
******* 11 780 [D loss: 0.208474, acc: 88.28%] [G loss: 8.223677]
******* 12 780 [D loss: 0.219742, acc: 90.62%] [G loss: 6.637879]
******* 13 780 [D loss: 0.256052, acc: 89.84%] [G loss: 5.693517]
******* 14 780 [D loss: 0.396984, acc: 81.25%] [G loss: 5.371883]
******* 15 780 [D lo

KeyboardInterrupt: ignored

In [ ]:
noise = np.random.normal(0, 1, (1,latent_dim))
gen_imgs = generator.predict(noise)

In [ ]:
gen_imgs = (gen_imgs + 1) / 2.0
plt.imshow(gen_imgs)

### **8) Making GIF**

In [ ]:
# Display a single image using the epoch number
# def display_image(epoch_no):
#   return PIL.Image.open('generated_images/%.8f.png'.format(epoch_no))

anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('generated_images/*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)